# Spark - Recommender System

**Imports**

In [28]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

**Start spark session and read data**

In [7]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [10]:
df = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [11]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [12]:
df.count()

1501

In [13]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



**Split data**

In [23]:
train_df, test_df = df.randomSplit([0.8, 0.2])

**Model**

In [24]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [25]:
model = als.fit(train_df)

**Prediction**

In [26]:
predictions = model.transform(test_df)

In [27]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      3|   1.0|    28|-0.34823284|
|      1|   1.0|    26|  1.3771621|
|      2|   1.0|    26|  1.8415248|
|      5|   2.0|    26| -0.9434276|
|      5|   2.0|    22|  1.2701911|
|      3|   1.0|     1|  0.7007606|
|      0|   1.0|    13| 0.02260065|
|      1|   4.0|    15|  0.8687047|
|      2|   1.0|    17|  0.8990648|
|      4|   1.0|     9|   2.356185|
|      3|   2.0|     8|  2.7019448|
|      4|   1.0|    23|-0.30082572|
|      4|   1.0|     7|  2.5713956|
|      0|   3.0|    10|   2.445042|
|      1|   3.0|    25|  1.0628293|
|      5|   1.0|    29|  1.9645233|
|      2|   4.0|    21|  3.1072564|
|      3|   1.0|    21|  2.6350176|
|      1|   1.0|    14|  0.7097839|
|      4|   1.0|    14|  2.2342968|
+-------+------+------+-----------+
only showing top 20 rows



**Evaluation model**

In [38]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='rmse')

In [40]:
rmse = evaluator.evaluate(predictions)
rmse

1.6466602456704689

**Recomemdations**

In [49]:
single_user = test_df.filter(test_df['UserId']==11).select('movieId', 'userId')

In [50]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     13|    11|
|     18|    11|
|     22|    11|
|     27|    11|
|     39|    11|
|     59|    11|
|     70|    11|
|     80|    11|
|     81|    11|
|     86|    11|
|     90|    11|
|     94|    11|
+-------+------+



In [51]:
recommendations = model.transform(single_user)

In [54]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     18|    11|  5.035202|
|     80|    11| 3.8331516|
|     39|    11| 3.6820476|
|     90|    11| 3.4830482|
|     22|    11| 3.4766424|
|     27|    11| 3.3375447|
|     94|    11|  2.776441|
|      6|    11| 2.4500823|
|     59|    11| 2.4404414|
|     70|    11| 2.1853714|
|     86|    11| 1.9046199|
|     13|    11|0.42249718|
|     81|    11|-1.2791772|
+-------+------+----------+

